In [1]:
import os
from dotenv import load_dotenv

# 加载环境变量（从 .env 文件）
load_dotenv()

# 如果环境变量未设置，设置默认值（仅用于测试）
if not os.getenv('POSTGRES_PASSWORD'):

    os.environ['POSTGRES_PASSWORD'] = 'grag_password'
if not os.getenv('API_PASSWORD'):
    os.environ['API_PASSWORD'] = 'admin123'
if not os.getenv('DEEPSEEK_API_KEY'):
    os.environ['DEEPSEEK_API_KEY'] = 'your_deepseek_api_key_here'

from knowledge.lightrag_wrapper import LightRAGWrapper
from models.model_manager import ModelManager



In [2]:
# 初始化（需要先配置好数据库和模型）
# 注意：模型配置已从 deepseek/local 改为 api/local
# - api: API 模型（DeepSeek），当前使用
# - local: 本地模型（Ollama），暂时禁用
model_manager = ModelManager()

# 验证模型配置
print(f"当前模型类型: {model_manager.get_current_model_type()}")
print(f"模型优先级配置: {model_manager.priority}")

lightrag = LightRAGWrapper(model_manager)

# 导入清洗后的文档
result = lightrag.ingest_from_json_file(
    'agent/tools/samples/google_generative-ai-docs_cleaned_docs.json'
)

print(f"✅ 成功导入 {result['total_documents']} 个文档")
print(f"数据源: {result['source']}")
print(f"仓库: {result['repo_url']}")

⚠️ API 模型不可用（健康检查失败）


当前模型类型: none
模型优先级配置: api


ERROR: [] PostgreSQL database,
sql:SELECT id FROM LIGHTRAG_DOC_STATUS WHERE workspace=$1 AND id = ANY($2),
params:{'workspace': '', 'ids': ['doc-f72416ec6c75b10c8f8bb362f2909084', 'doc-6696e804b17c11820144f2537ea15533', 'doc-8363540de38faafdf47499aab5ddb4f5']},
error:'NoneType' object has no attribute 'query'
❌ 导入文档失败: 'NoneType' object has no attribute 'query'
💡 提示: 请检查模型配置是否正确，确保 API 密钥已设置且模型可用
💡 提示: 检查 DEEPSEEK_API_KEY 环境变量或 config.yaml 中的 models.api.api_key


AttributeError: 'NoneType' object has no attribute 'query'

In [ ]:
import os
print(f"HTTP_PROXY: {os.environ.get('HTTP_PROXY', '未设置')}")
print(f"HTTPS_PROXY: {os.environ.get('HTTPS_PROXY', '未设置')}")
print("POSTGRES_DATABASE:", os.getenv('POSTGRES_DATABASE'))
print("POSTGRES_USER:", os.getenv('POSTGRES_USER'))
print("POSTGRES_DB:", os.getenv('POSTGRES_DB'))

In [ ]:
from config.config_manager import get_config

config = get_config()
db_config = config.get_database_config()

print("配置中的数据库名:", db_config.get('database'))
print("配置中的用户名:", db_config.get('user'))

In [ ]:
# 验证 lightrag 包的导入
import sys

# 方法 1: 直接尝试导入
try:
    import lightrag
    print(f"✅ import lightrag 成功")
    print(f"   模块位置: {lightrag.__file__}")
    print(f"   模块属性: {dir(lightrag)[:10]}")
except ImportError as e:
    print(f"❌ import lightrag 失败: {e}")

# 方法 2: 检查包信息
try:
    import pkg_resources
    dist = pkg_resources.get_distribution('lightrag-hku')
    print(f"\n✅ 包信息:")
    print(f"   包名: {dist.project_name}")
    print(f"   版本: {dist.version}")
    print(f"   位置: {dist.location}")
    
    # 检查包中的模块
    print(f"\n   包中的文件:")
    for item in dist.get_metadata_lines('RECORD')[:5]:
        print(f"     {item}")
except Exception as e:
    print(f"❌ 无法获取包信息: {e}")

# 方法 3: 检查实际导入
try:
    from lightrag import LightRAG
    print(f"\n✅ from lightrag import LightRAG 成功")
except ImportError as e:
    print(f"\n❌ from lightrag import LightRAG 失败: {e}")
    print(f"   错误详情: {type(e).__name__}: {e}")

In [ ]:
# 测试模型配置
from config.config_manager import get_config

config = get_config()
api_config = config.get_model_config("api")
local_config = config.get_model_config("local")

print("API 模型配置:")
print(f"  - API Key 已设置: {'是' if api_config.get('api_key') else '否'}")
print(f"  - Base URL: {api_config.get('base_url', '未设置')}")
print(f"  - Model Name: {api_config.get('model_name', '未设置')}")

print("\n本地模型配置（暂时禁用）:")
print(f"  - Base URL: {local_config.get('base_url', '未设置')}")
print(f"  - Model Name: {local_config.get('model_name', '未设置')}")

print("\n模型切换策略:")
model_switch = config.get("model_switch", {})
print(f"  - Priority: {model_switch.get('priority', '未设置')}")
print(f"  - Fallback to API: {model_switch.get('fallback_to_api', False)}")


In [ ]:
import sys
import subprocess

# 检查当前 Python 环境
print(f"当前 Python: {sys.executable}")
print(f"是否在 .venv: {'.venv' in sys.executable}")

# 检查 lightrag-hku 是否在当前环境中
result = subprocess.run(
    [sys.executable, "-m", "pip", "show", "lightrag-hku"],
    capture_output=True,
    text=True
)
print(f"\n包信息检查:")
print(result.stdout if result.returncode == 0 else result.stderr)

# 尝试导入
try:
    import lightrag
    print(f"\n✅ lightrag 模块导入成功")
    print(f"   位置: {lightrag.__file__}")
except ImportError as e:
    print(f"\n❌ 导入失败: {e}")
    print(f"\n建议:")
    print(f"1. 重启 Jupyter kernel")
    print(f"2. 确保 kernel 使用: {sys.executable}")